## Notebook component to populate a Redshift cluster with our data

In [112]:
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import os

### Access credentials securely

In [2]:
load_dotenv(".env")

user = os.environ.get("REDSHIFT_USERNAME")
password = os.environ.get("REDSHIFT_PASSWORD")
host = os.environ.get("REDSHIFT_HOST")
iamrole = os.environ.get("IAM_role")

### Form connection string

Note, this assumes we added

```bash
poetry add sqlalchemy-redshift

```

In [3]:
%pip install redshift-connector "sqlalchemy<2" --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
url = URL.create(
    drivername="redshift+redshift_connector",
    username=user,
    password=password,
    host=host,
    port=5439,
    database="dev",
)

engine = create_engine(url)

### Load JupySQL extension

In [5]:
%reload_ext sql

Found pyproject.toml from '/Users/macpro/Documents/GitHub/automate-elt-github'

Settings changed:

Config,value
displaycon,False
feedback,True
autopandas,True
named_parameters,True


### Use the engine to initialize access to our Redshift via the alias `redshift`

In [6]:
%sql engine --alias redshift

### Add data to S3 bucket

In [115]:
! aws s3 cp expanded_data/account.csv s3://ploomber-redshift-data/nyc-taxi/account.csv
! aws s3 cp expanded_data/district.csv s3://ploomber-redshift-data/nyc-taxi/district.csv

6421.41s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


upload: expanded_data/account.csv to s3://ploomber-redshift-data/nyc-taxi/account.csv


6433.38s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


upload: expanded_data/district.csv to s3://ploomber-redshift-data/nyc-taxi/district.csv


### Create table `account` and table `district` from data in S3 bucket

Ensure you set up your access key and secret access keys!

In [116]:
%%sql
DROP TABLE IF EXISTS account CASCADE;
DROP TABLE IF EXISTS district CASCADE;

""


In [117]:
%%sql
CREATE TABLE IF NOT EXISTS account
(account_id INT,
 district_id INT,
 frequency VARCHAR(50),
 date VARCHAR(50));

 COPY account
FROM 's3://ploomber-redshift-data/nyc-taxi/account.csv'
IAM_ROLE '{{iamrole}}'
IGNOREHEADER 1
CSV;

""


In [118]:
%%sql
CREATE TABLE IF NOT EXISTS district
(district_id INT,
district_name VARCHAR(50),
region VARCHAR(50),
no_of_inhabitants BIGINT ,
no_of_municipalities_lt_499 INT,
no_of_municipalities_500_1999 INT,
no_of_municipalities_2000_9999 INT,
no_of_municipalities_gt_10000 INT,
no_of_cities INT,
ratio_of_urban_inhabitants DOUBLE PRECISION,
average_salary DOUBLE PRECISION,
unemployment_rate_95 DOUBLE PRECISION,
unemployment_rate_96 DOUBLE PRECISION,
no_of_entrepreneurs_per_1000_inhabitants DOUBLE PRECISION,
no_of_committed_crimes_95 INT,
no_of_committed_crimes_96 INT);

 
COPY district
FROM 's3://ploomber-redshift-data/nyc-taxi/district.csv'
IAM_ROLE '{{iamrole}}'
IGNOREHEADER 1
CSV;

""


## Profile the data

In [123]:
%sqlcmd profile -t account

,account_id,district_id,frequency,date
count,4500,4500,4500,4500
unique,4500,77,3,1535
mean,2786.0000,37.0000,nan,951654.0000
min,1,1,nan,
max,11382,77,nan,


In [124]:
%sqlcmd profile -t district

,district_id,district_name,region,no_of_inhabitants,no_of_municipalities_lt_499,no_of_municipalities_500_1999,no_of_municipalities_2000_9999,no_of_municipalities_gt_10000,no_of_cities,ratio_of_urban_inhabitants,average_salary,unemployment_rate_95,unemployment_rate_96,no_of_entrepreneurs_per_1000_inhabitants,no_of_committed_crimes_95,no_of_committed_crimes_96
count,76,76,76,76,76,76,76,76,76,76,76,76,76,76,76,76
unique,76,76,8,76,52,36,17,6,11,69,75,70,72,44,75,75
mean,38.0000,nan,nan,135083.0000,49.0000,24.0000,6.0000,1.0000,6.0000,63.2276,9042.9737,3.1193,3.7446,116.0263,4850.0000,5079.0000
min,1,nan,nan,45714,0,0,0,0,1,33.9,8110.0,0.29,0.43,81.0,818,888
max,77,nan,nan,1204953,151,70,20,5,11,100.0,12541.0,7.34,9.4,167.0,85677,99107


### Join tables

In [128]:
%%sql
CREATE TABLE IF NOT EXISTS account_district AS
SELECT 
    account.account_id,
    account.district_id,
    account.frequency,
    account.date,
    district.district_name,
    district.region,
    district.no_of_inhabitants,
    district.no_of_municipalities_lt_499,
    district.no_of_municipalities_500_1999,
    district.no_of_municipalities_2000_9999,
    district.no_of_municipalities_gt_10000,
    district.no_of_cities,
    district.ratio_of_urban_inhabitants,
    district.average_salary,
    district.unemployment_rate_95,
    district.unemployment_rate_96,
    district.no_of_entrepreneurs_per_1000_inhabitants,
    district.no_of_committed_crimes_95,
    district.no_of_committed_crimes_96
FROM account
INNER JOIN district 
    ON account.district_id = district.district_id;


RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(redshift_connector.error.ProgrammingError) {'S': 'ERROR', 'C': '42601', 'M': 'syntax error at or near "AS"', 'P': '45', 'F': '/home/ec2-user/padb/src/pg/src/backend/parser/parser_scan.l', 'L': '732', 'R': 'yyerror'}
[SQL: CREATE TABLE IF NOT EXISTS account_district AS
SELECT 
    account.account_id,
    account.district_id,
    account.frequency,
    account.date,
    district.district_name,
    district.region,
    district.no_of_inhabitants,
    district.no_of_municipalities_lt_499,
    district.no_of_municipalities_500_1999,
    district.no_of_municipalities_2000_9999,
    district.no_of_municipalities_gt_10000,
    district.no_of_cities,
    district.ratio_of_urban_inhabitants,
    district.average_salary,
    district.unemployment_rate_95,
    district.unemployment_rat

### Clean up data

In [114]:
! aws s3 rm s3://ploomber-redshift-data/nyc-taxi/account.csv
! aws s3 rm s3://ploomber-redshift-data/nyc-taxi/district.csv

6391.48s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


delete: s3://ploomber-redshift-data/nyc-taxi/account.csv


6403.46s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


delete: s3://ploomber-redshift-data/nyc-taxi/district.csv


### Delete tables

In [ ]:
%%sql
DROP TABLE account;
DROP TABLE district;